# 1. Парсер vk

Ознакомиться со способами взаимодействия с Вконтакте можно по [ссылке](https://vk.com/dev/manuals) на отличную документацию. Для того, чтобы получить доступ к выгрузке данных, нужно пройти ряд бюрократических процедур.

Первая такая процедура заключается в создании своего приложения. Для этого переходим по [ссылке](http://vk.com/editapp?act=create) и проходимся по необходимым шагам:

<img align="center" src="pictures/app_creation_1.png" height="600" width="600">

После подтверждения своей личности по номеру телефона, попадаем на страницу свежесозданного приложения
<img align="center" src="pictures/app_creation_2.png" height="600" width="600">

Слева нам будем доступна вкладка с настройками, перейдя в неё мы увидим все необходимые нам для работы с приложением параметры:
<img align="center" src="pictures/app_creation_3.png" height="600" width="600">


Для работы с частью методов API этого вполне достаточно (обычно в заголовке такого метода стоит соответствующая пометка). Для части методов, используемых нами, может понадобиться ключ доступа.

Для того, чтобы получить его, необходимо сделать ещё пару странных манипуляций:

Переходим по ссылке вида (на месте звездочек должен стоять ID созданного вами приложения):

> https://oauth.vk.com/authorize?client_id=**********&scope=14&redirect_uri=https://oauth.vk.com/blank.html&display=page&v=5.16&response_type=token

<img align="center" src="pictures/app_creation_4.png" height="600" width="600">

В итоге по этому запросу будет сформирована ссылка следующего вида:
> https://oauth.vk.com/blank.html#access_token=25b636116ef40e0718fe4d9f382544fc28&expires_in=86400&user_id=*******

Первый набор знаков - access token, т.е. маркер доступа. Вторая цифра (expires_in=) время работы маркера доступа в секундах (одни сутки). По истечению суток нужно будет получить новый маркер доступа. Последняя цифра (user_id=) ваш ID Вконтакте. Нам в дальнейшем понадобится маркер доступа. Для удобства сохраним его в отдельном файле или экспортируем в глобальную область видимости. В целях безопасности ваших данных не стоит нигде светить токенами и тем более выкладывать их в открытый доступ.

Обратите внимание на ссылку, по которой мы делали запрос на предоставление токена. Внутри неё находится странный параметр scope=14. Эта загадочная цифра есть ничто иное, как права доступа к социальной сети. Подробнее познакомиться с взаимно-однозначным соответствием между числами и правами можно в документации. Например, если мы хотим получить доступ к друзьям, фото и аудио, мы подставим в scope цифру 2+4+8=14.

In [1]:
# Загружаем необходимые библиотеки

import datetime                  # Пакет для работы с временными форматами
import pickle                    # Пакет для подгрузки данных специфического для питона формата
import requests                  # Пакет для скачки данных из этих ваших интернетов
import pandas as pd              # Пакет для работы с таблицами
import numpy as np               # Пакет для работы с векторами и матрицами
import matplotlib.pyplot as plt  # Пакет для строительства графиков
import time            # Пакет для работы со временем. Например, помогает ставить заглушки 
                       # time.sleep(секунды), необходимые для того что ВК не банил нашего
                       # сборщика данных из-за слишком частых запросов
        
# Пакет для красивых циклов. При желании его можно отключить. Тогда из всех циклов придётся 
# удалять команду tqdm_notebook.
from tqdm import tqdm_notebook   # подробнее: https://github.com/tqdm/tqdm

In [6]:
# мой номер странички
my_user_id = 371834160
# версия используемого API
version = '5.103' 

# подгружаем токен
with open('C:/Users/Asus/GitHab_repos/vk_at.txt') as f:
    token = f.read()

In [7]:
def vkDownload(method, parameters, token=token, version=version):
    """
        Возвращает результат запроса по методу
        
        method: string
            метод из документации, который хотим использовать
            
        parameters: string
            параметры используемого метода
            
        token: string
            токен Oauth доступа
        
        version: string
            версия API
    """
    
    
    # составляем ссылку
    url = 'https://api.vk.com/method/{method}?{parameters}&access_token={token}&v={version}'
    url = url.format(method=method, parameters=parameters, token=token, version=version)
    # запрашиваем ссылку и переводим в json (словарь)
    response = requests.get(url).json()
    
    
    return response

vkDownload('users.get','user_ids=371834160')    

{'response': [{'id': 371834160,
   'first_name': 'Sergei',
   'last_name': 'Korneev',
   'is_closed': False,
   'can_access_closed': True}]}

In [8]:
response['response']

NameError: name 'response' is not defined

In [3]:
def makeBatch(uids, size=25):
    """
        Возвращает лист листов из пользователей, батчами по size человек 
        По умолчанию size=25
    """
    batches = [uids[i:i + size] for i in range(0,len(uids),size)]
    
    return batches 

## 1.1 Подписчики каждой группы для жанра

In [5]:
music_style = 'rhh'

target_groups={
    'lizer' : '117382436',
    'morgenshtern' : '123675921',
    'face' : '95470601',
    'lj' : '41623203'
    }

In [7]:
# слоаврь группа-юзеры
group_users_dct = {}

Перед тем, как сохранять юзеров, надо профильтровать их на предмет существования... 

In [8]:
def checkExisting(uids):
    """
        Проверяет существование id и оставляет только не удаленные 
    """
    
    cur_inf = vkDownload('users.get','user_ids={}'.format(','.join(str(e) for e in uids)))
    cur_inf = cur_inf['response']
    clean_members = [us_inf['id'] for us_inf in cur_inf if us_inf.get('deactivated','real') == 'real']
    return clean_members


def getGroupMembers(group_id):
    """
        Возвращает список всех пользователей данной группы
        Итерации идут батчами по 1000 пользователей
        
        group_id: string
            идентификатор группы (ссылка)
    """
    
    # Узнаём число запросов, которое надо сделать 
    count = vkDownload('groups.getMembers','group_id=' + group_id)['response']['count']
    
    # сэплируем число подписчиков, которые мы хотим вытащить
    # берём либо 10%, либо 100 000, смотря что больше
    n_sampled = int(max(0.1 * count, min(count, 100000)))
    print("Общее число пользователей: {}\nСэмплированное число пользователей: {}".format(count, n_sampled))
    
    # выясняем, сколько запросов нам понадобится
    n = np.linspace(0, count/1000, num=n_sampled/1000).astype(int)
    
    #n = int(np.ceil(count/1000))  
    
    # вектор, где мы будем хранить id пользователей
    members = []     
    
    for i in tqdm_notebook(n): 
        try:
            # при помощи метода groups.getMembers получаем пользователей группы
            current_members = vkDownload('groups.getMembers','group_id='+group_id+'&offset='+str(1000*i))
            current_members = current_members['response']['items']
        
            # проверим реально ли существуют все юзеры из списка выше
            for i in range(0,1000,200):
                time.sleep(0.3)
                members.extend(checkExisting(current_members[i:i+200]))

            # перед следующим запросом немножко подождем
            time.sleep(0.3)
        except Exception as e:
            print(e)
            next
        
    return members

In [9]:
# по каждому id группы сохраняем подписчиков 
for k,v in tqdm_notebook(target_groups.items()):
    try:
        members = getGroupMembers(target_groups[k]) 
        print(k,':',len(members))
        group_users_dct[k] = members
    except KeyboardInterrupt:
        break

Общее число пользователей: 458192
Сэмплированное число пользователей: 100000


C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.


lizer : 97394
Общее число пользователей: 1294128
Сэмплированное число пользователей: 129412


morgenshtern : 117586
Общее число пользователей: 939281
Сэмплированное число пользователей: 100000


face : 84548
Общее число пользователей: 1463981
Сэмплированное число пользователей: 146398


lj : 121935



In [12]:
# Сохраняем словарик для текущего жанра 
with open('{}_group_users'.format(music_style), 'wb') as f:
    pickle.dump(group_users_dct, f)

In [9]:
with open('{}_group_users'.format(music_style), 'rb') as f:
    group_users_dct = pickle.load(f)

NameError: name 'music_style' is not defined

In [7]:
# первые 5 фанатов Элджея:

group_users_dct['morgenshtern'][:5]

[3330, 3450, 8015, 17212, 30488]

In [1]:
def vk_download(method,parameters,token = token,version=version):
    url = 'https://api.vk.com/method/{method}?{parameters}&access_token={token}&v={version}'
    url = url.format(method=method, parameters=parameters, token=token, version=version)
#     response = 
    return requests.get(url).json()

morgenshtern_fans_bio = { }   

# что скачиваем
fields = 'photo_id,sex,bdate,city,country,home_town,education,universities,schools,status,followers_count,occupation,interests,movies,tv,books,games'

for id_ in tqdm_notebook(group_users_dct['morgenshtern'][:1000]): 
    time.sleep(0.4)   # Не забываем отдохнуть между запросами, чтобы вк не злился! 
    try:
        res = vk_download('users.get','user_ids='+str(id_)+'&fields=' + fields)['response'][0]
       
        # закидываем новую информацию в словарик 
        morgenshtern_fans_bio[id_] = res   
    except:
        # Если скачка не удалась, выведем сообщение об ошибке
        # (По идее, мы хорошо обработали до этого всех юзеров и ошибок не будет)
        print('Ощибка с ', id_)


NameError: name 'token' is not defined

### Сохранить на диск

In [149]:
with open('lizer_fans_bio', 'wb') as f:
    pickle.dump(lizer_fans_bio, f)

### Открыть

In [97]:
# мой номер странички
my_user_id = 371834160
# версия используемого API
version = '5.103' 

# подгружаем токен
with open('C:/Users/Asus/GitHab_repos/vk_at.txt') as f:
    token = f.read()

In [99]:
def vkDownload(method, parameters, token=token, version=version):
    """
        Возвращает результат запроса по методу
        
        method: string
            метод из документации, который хотим использовать
            
        parameters: string
            параметры используемого метода
            
        token: string
            токен Oauth доступа
        
        version: string
            версия API
    """
    
    
    # составляем ссылку
    url = 'https://api.vk.com/method/{method}?{parameters}&access_token={token}&v={version}'
    url = url.format(method=method, parameters=parameters, token=token, version=version)
    # запрашиваем ссылку и переводим в json (словарь)
    response = requests.get(url).json()
    
    
    return response

In [59]:
import pickle
with open('lj_fans_bio', 'rb') as f:
    lj_fans_bio = pickle.load(f)

In [60]:
lj_fans_ids = []

for i in lj_fans_bio:
    lj_fans_ids.append(i)

In [62]:
lj_fans_photo_ids = []

for i in lj_fans_ids:
    for k,v in lj_fans_bio[i].items():
        if k == 'photo_id':
            lj_fans_photo_ids.append(v)

In [98]:
# lj_fans_photo_ids

In [105]:
vkDownload('likes.getList','photo628_456239032')

{'error': {'error_code': 100,
  'error_msg': 'One of the parameters specified was missing or invalid: type is undefined',
  'request_params': [{'key': 'photo628_456239032', 'value': ''},
   {'key': 'v', 'value': '5.103'},
   {'key': 'method', 'value': 'likes.getList'},
   {'key': 'oauth', 'value': '1'}]}}

In [ ]:


vkDownload('users.get','user_ids=371834160')   

In [125]:
url = "https://api.vk.com/method/users.get?user_ids=6045249&access_token="+str(token)
response = requests.get(url) 
response

{'error': {'error_code': 8,
  'error_msg': 'Invalid request: v is required. Version param should be passed as "v". "version" param is invalid and not supported.',
  'request_params': [{'key': 'user_ids', 'value': '6045249'},
   {'key': 'method', 'value': 'users.get'},
   {'key': 'oauth', 'value': '1'}]}}

In [122]:
def vk_download(method, parameters, token = token):
    url = 'https://api.vk.com/method/'+method+'?'+parameters+'&access_token='+str(token)
    response = requests.get(url) 
    return(response.json())

In [123]:
def Get_Wall(this_id):
    
    # Качаем со стены все записи!
    chto_on_postit = [ ]
    for i in range(10,100):
        # Качаем внеочередные 100 записей со стены.
        time.sleep(0.4) # Не забываем немного вздремнуть 
        # Первый элемент всегда число записей, игнорируем его 
        wall = vk_download('wall.get', 'owner_id=' + str(this_id) +'&count=100&offset='+str(i))['response'][1:]
        chto_on_postit.extend(wall)
        
    # В файлике chto_on post

In [124]:
a = Get_Wall('628')

KeyError: 'response'

## 1.2 Файл с комментами каждой группы жанра

Скачиваем с группы все посты.

In [18]:
def getPosts(group_id):
    """
        Качает по группе список всех постов, которые в 
        ней есть и возвращает список из их id 
    """
    
    post_ids = [ ] # для сбора id
    
    # Выясним сколько всего в группе постов 
    n = vkDownload('wall.get','owner_id=-{}'.format(group_id))['response']['count']
    print("В группе {} должно быть {} постов".format(group_id,n))
    
    for i in tqdm_notebook(range(0, n + 100, 100)):
        time.sleep(0.3)
        
        wall = vkDownload('wall.get','owner_id=-{}&count=100&offset='.format(group_id)+str(i))['response']['items']
        # если захочется отделить комменты группы от других, есть поле from_id 
        post_ids.extend([item['id'] for item in wall])
        
    return post_ids


def prepareComments(group_id, posts):
    """
        Мы будем скачивать 25 запросами за раз, эта функция готовит запросы 
        к последущему использованию, понимая сколько комментов под каким постом, 
        выкидывая посты с нулевым количеством комментов и делая тройки (группа, пост, оффсет)
    """

    triple_comments = [ ] # для троек (group_id, post_id, offset)

    # делаем из всех постов батчи по 25 штук, чтобы было быстрее
    posts_batch = makeBatch(posts)

    for batch in tqdm_notebook(posts_batch):
        time.sleep(0.4)

        # готовим запрос для комметов из батча, который выяснит число комментов под постом
        begin = 'https://api.vk.com/method/execute?code=return['
        end = '];&access_token='+token+'&v=5.78'
        middle = ''

        # для каждого поста из батча используем метод wall.getComments
        for bt in batch:            
            middle += 'API.wall.getComments({"owner_id":"-' + str(group_id) + '","post_id":"' + str(bt) + '"}),'

        # делаем запрос для текущих 25 комментов
        requests.get(begin + middle[:-1] + end)

        # выясняем сколько всего комментов под постом 
        s = requests.get(begin + middle[:-1] + end)
        
        # по добытой информации готовим запросы на будущее
        try:
            for post,item in zip(batch, s.json()['response']):
                if item is False:
                    print('False в посте {} группы {}'.format(post, group_id))
                    continue

                n = item['count']

                # пост с нулевым числом комментов игнорим
                if n == 0:
                    continue 
                else:
                    # для каждой сотни комментов из-под поста с комментами будет свой запрос для добычи
                    for i in range(0, n, 100):
                        triple_comments.append((group_id, post, i))
        except:
            print("broken batch")
            continue
    return triple_comments


def getComments(triple_comments):
    """
        Делит элементы вектора вида (group_id, post_id, offset), 
        на батчи по 15 штук и качает комменты аки демон
    """
    
    comments = [ ] # для сбора комментов 

    # делаем батчи 
    precom_batch = makeBatch(triple_comments, 15)

    for batch in tqdm_notebook(precom_batch):
        time.sleep(0.4)
        # клепаем запрос 
        begin = 'https://api.vk.com/method/execute?code=return['
        end = '];&access_token='+token+'&v=5.78'
        middle = ''

        for bt in batch:            
            middle += 'API.wall.getComments({"owner_id":"-' + str(bt[0]) + '","post_id":"' + str(bt[1]) + '",\
            "need_likes":"1","preview_length":"0","count":"100","offset":"'+ str(bt[2])+'"}),'

        # делаем запрос для текущих 20 комментов
        s = requests.get(begin + middle[:-1] + end)

        # распаковываем добытое добро
        
        cur_com = [ ]
        try:
            for item, post in zip(s.json()['response'], batch):
                for jtem in item['items']:
                    jtem['likes'] = jtem['likes']['count']
                    jtem['post_id'] = post[1]
                    jtem['group_id'] = post[0]
                    cur_com.append(jtem)
        except:
            pass
        comments.extend(cur_com) 
        
    return comments    

In [19]:
group_names = {v:k for k,v in target_groups.items()}

In [21]:
unique_comments = { }

for name in target_groups:
    try:
        # качаем посты 
        group_id = target_groups[name]
        posts = getPosts(group_id)

        print('Из группы {} имени {} скачалось {} постов'.format(target_groups[name], name, len(posts)))

        # готовим батчи из запросов для комментов
        triple_comments = prepareComments(group_id, posts)

        print('Число запросов: {}'.format(len(triple_comments)))

        # качаем комменты
        comments = getComments(triple_comments)

        setlen = len(set([str(comment['post_id']) + '_' + str(comment['id']) for comment in comments]))
        print('Число комментов: {}, из них уникальных: {}'.format(len(comments), setlen ))

        # смотрим на десяток последних комментов 
        for comment in comments[-10:]:
            print(comment['post_id'],comment['id'])
            print(comment['text'])
            print('--------------------')

        # записываем накачаное добро в словарик
        unique_comments[name] = comments

        # Сохраняем ну хоть что-нибудь... 
        with open('../data/{}_group_comments_{}'.format(music_style, group_names[group_id]), 'wb') as f:
            pickle.dump(comments, f)

        print('====================================================================')
    except KeyboardInterrupt:
        break

В группе 55072656 должно быть 704 постов
